In [ ]:
%config IPCompleter.greedy=True

import yaml
import psycopg2
import os
import os.path
import sys

# Set the path
home_dir = os.getcwd()
credential_dir = os.path.join('../../config')

def create_pgconn(credentials_yaml):
    with open(credentials_yaml) as f:
        configs = yaml.load(f)
    try: 
        conn = psycopg2.connect("dbname='{}' user='{}' host='{}' password='{}'".format(
            configs['DB_name'],
            configs['user'],
            configs['host'],
            configs['password']))
    except Exception as e: 
        print("Error connecting to db.")
        raise e
    conn.set_client_encoding('latin_1')
    cur = conn.cursor()
    cur.execute("SET ROLE " + configs['role'])
    return conn

In [ ]:
# Setting up a connection to the server

credentials_yaml = os.path.join(credential_dir, 'db_creds.yml') #example file on hitchikers repo
conn = create_pgconn(credentials_yaml)

def sql(query, conn=conn):
    return pd.read_sql(query, conn)

In [ ]:
# To rename columns, you need to find out the unique column names in the original files
cur = conn.cursor()
cur.execute("""select distinct(orginal_col_name) from raw.column_mapping where table_name ~ 'raw."6.*';""")
[print(x[0][1:-1]) for x in cur.fetchall()];

In [ ]:
# Using the manually created 6_column_mapping.csv, a dictionary of old column name to new column name can be created
with open('../../../../garfield/6_column_mapping.csv', 'r') as fil:
    col_map = fil.read()
col_names=dict()
col_types=dict()
union = []
for pair in col_map.split('\n'):
    split_pair = pair.split(',')
    col_names[split_pair[0]] = split_pair[1]
    col_types[split_pair[1]] = split_pair[2]
    union += [split_pair[1]]
union=set(union)
# NOT SURE THIS IS APPROPRIATE
# # The set union will be used to determine the number and name of columns in the joined table. 
# # Instead of trusting the year column in the .csv (which is called raw_year), we will replace the year according to the file structure (and call it year)
# union.remove('raw_year')
# union.add('year')

In [ ]:
# This creates a list of tables that we can iterate over to rename columns and join
cur = conn.cursor()
cur.execute("""select distinct(table_name) from raw.column_mapping where table_name ~ 'raw."6.*';""")
table_list = [x[0] for x in cur.fetchall()]
# table_list = []
# for table in old_table_list:
#     if '2014' not in table:
#         table_list += [table]

# Preprocessing

## Each file renamed columns

In [ ]:
# # Iterates over all tables in raw (with columns a0, a1, a2...), creating a table in preprocessing (with columns nie, dpto_code_ce, year...)
cur = conn.cursor()
for table in table_list:
    print(table)
    new_table = table.replace('raw', 'preproc')
    cur.execute("""drop table if exists %s;""" %new_table)
#     cur.execute("""select mapped_col_name, orginal_col_name from raw.column_mapping where table_name='%s' order by mapped_col_name;""" %table)
#     col_pairs = cur.fetchall()
#     raw_cols = []
#     renamed_cols = []
#     for col_pair in col_pairs:
#         raw_cols += [col_pair[0]]
#         renamed_cols += [col_dict[col_pair[1][1:-1]]]
#     cmnd = ' varchar, '.join(renamed_cols) + ' varchar'
#     cur.execute("""create table %s (%s);""" %(new_table, cmnd))
#     cur.execute("""insert into %s (%s) select %s from %s;""" %(new_table, ','.join(renamed_cols), ','.join(raw_cols), table))
conn.commit()

## Cleaning

In [ ]:
# # Iterate over all tables 
# #     Adding a year column that uses the file name rather than the one inside the file
# cur = conn.cursor()
# for table in table_list:
#     new_table = table.replace('raw', 'preproc')
#     table_name = new_table.split('.')[1]#.replace('"', '')
#     cur.execute("""ALTER TABLE %s ADD COLUMN year varchar DEFAULT %s;""" %(new_table, (table_name[3:7])))
# conn.commit()

## Joined table

In [ ]:
cur = conn.cursor()
cmnd = ' varchar, '.join(union) + ' varchar'  # This creates a list of column names that will be used in the joined table
cur.execute("""drop table if exists preproc."6_joined";""")
cur.execute("""create table if not exists preproc."6_joined" (%s);""" %cmnd)
conn.commit()

In [ ]:
# # Iterates over all the preprocessing tables and inserts them into the joined table
# # cur = conn.cursor()
# for table in table_list:
#     print(table)
#     new_table = table.replace('raw', 'preproc')
#     cur.execute("""select orginal_col_name from raw.column_mapping where table_name ='raw."6_2014_MEDIA_media"';""")
#     old_cols = cur.fetchall()
#     break
#     renamed_cols = []
#     for col_pair in col_pairs:
#         renamed_cols += [col_dict[col_pair[1][1:-1]]]
#     col_str = ', '.join(renamed_cols)
# #     col_str = col_str.replace('raw_year', 'year')
#     cur.execute("""insert into preproc."6_joined" (%s) select %s from %s;""" %(col_str, col_str, new_table))
# conn.commit()

In [ ]:
# Iterates over all tables in raw (with columns a0, a1, a2...), inserting into a joined preprocessing table (with columns nie, dpto_code_ce, year...)
cur = conn.cursor()
for table in table_list:
    print(table)
    new_table = table.replace('raw', 'preproc')
    cur.execute("""select mapped_col_name, orginal_col_name from raw.column_mapping where table_name='%s' order by mapped_col_name;""" %table)
    col_pairs = cur.fetchall()
    raw_cols = []
    renamed_cols = []
    for col_pair in col_pairs:
        raw_cols += [col_pair[0]]
        renamed_cols += [col_names[col_pair[1][1:-1]]]
    cmnd = ' varchar, '.join(renamed_cols) + ' varchar'
    cur.execute("""insert into preproc."6_joined" (%s) select %s from %s;""" %(','.join(renamed_cols), ','.join(raw_cols), table))
conn.commit()

In [ ]:
# Changing the column types and removing empty strings
cur = conn.cursor()
for col, col_typ in col_types.items():
    cur.execute("""ALTER TABLE preproc."6_joined" ALTER COLUMN %s TYPE %s using NULLIF(%s, '')::%s;""" %(col, col_typ, col, col_typ))
conn.commit()

In [ ]:
# Changing the grados column into a simple numerical key for simpler operations
grados_dict = {'1B PRIMER AÑO DE BACHILLERATO':10,
'01 PRIMER GRADO':1,
'07 SEPTIMO GRADO':7,
'06 SEXTO GRADO':6,
'05 QUINTO GRADO':5,
'04 CUARTO GRADO':4,
'03 TERCER GRADO':3,
'02 SEGUNDO GRADO':2,
'08 OCTAVO GRADO':8,
'09 NOVENO GRADO':9,
'2B SEGUNDO AÑO DE BACHILLERATO':11,
'3B TERCER AÑO DE BACHILLERATO':12,
'5P PARVULARIA 5 AÑOS':-1,
'6P PARVULARIA 6 AÑOS':0,
'4P PARVULARIA 4 AÑOS':-2,
'4B CUARTO AÑO DE BACHILLERATO':13,
'S2 SECCIÓN 2 AÑO':11,
'S1 SECCIÓN 1 AÑO':10}
cmnd = ''
for key, value in grados_dict.items():
    cmnd += """WHEN '%s' THEN %d """ %(key, value)

In [ ]:
# Changing the types of the joined table and adding a grade_code column
cur = conn.cursor()
new_table = 'preproc."6_joined"'
table_name = new_table.split('.')[1]
cur.execute("""ALTER TABLE %s ADD COLUMN grado_code numeric""" %new_table)
cur.execute("""ALTER TABLE %s ALTER COLUMN status TYPE bool
USING CASE status 
WHEN 'activo' THEN '1'::bool
WHEN 'inactivo' THEN '0'::bool
when '1' then '1'::bool
when '0' then '0'::bool
END;""" %new_table)
cur.execute("""ALTER TABLE %s ALTER COLUMN birth_date TYPE DATE using to_date(birth_date, 'DD-MM-YY');""" %new_table)
cur.execute("""ALTER TABLE %s ALTER COLUMN grado_code TYPE numeric using case grado %s END;""" %(new_table, cmnd))
cur.execute("""ALTER TABLE %s ALTER COLUMN gender TYPE char(1)
USING CASE gender 
WHEN 'Masculino' THEN 'M'
WHEN 'Femenino' THEN 'F'
END;""" %new_table)
conn.commit()

# Cleaning

In [ ]:
# TODO
#     Detect input errors
#         Unique school IDs
#         Check if a student is missing one year but appears the next in the following grade (so he's not a dropout)

In [ ]:
# Checking that all the school codes are unique
# First select all distinct combinations of school code, name and municipio code. Then count the numer of names given the same schoold and municipio code
cur = conn.cursor()
cur.execute("""with distinct_all as (select distinct ce_code, ce_name, munic_code_ce from preproc."6_joined")
select count(*), ce_code, munic_code_ce from distinct_all group by ce_code, munic_code_ce order by 1 desc;""")
print(cur.fetchall())
# From 2015 to 2018 there is only 1 that has two names and it has the ce_code 11682
# select ce_name from preproc."6_joined" where ce_code='11682' group by ce_name;
# Hence school codes are unique from 2015 to 2018

In [ ]:
# A first cleaning step is to simply exclude NULL and 0 NIEs
cur = conn.cursor()
cur.execute("""select count(*) from preproc."6_joined" where id isnull or id='0';""")
print(cur.fetchall())
cur.execute("""select * into preproc."6_cleaned" from preproc."6_joined" where id is not null and id!='0';""")
conn.commit()

In [ ]:
# Further cleaning steps will be to infer missing values by checking grades before and after

# Moving to staging

In [ ]:
# First need to find distinct combinations of nie, year
cur = conn.cursor()
cur.execute("""select distinct nie, year into cleaned.distinct_nie_year from cleaned.student_registration;""")
conn.commit()

In [ ]:
# Students remaining in or leaving the school system year per year
cur = conn.cursor()
cur.execute("""select a.nie, a.year into cleaned.remain_in_system from cleaned.distinct_nie_year as a left join cleaned.distinct_nie_year as b on a.nie=b.nie and b.year=a.year+1 where b.nie is not null;""")
cur.execute("""select a.nie, a.year into cleaned.left_system from cleaned.distinct_nie_year as a left join cleaned.distinct_nie_year as b on a.nie=b.nie and b.year=a.year+1 where b.nie is null;""")
conn.commit()

In [ ]:
# Setting the remains_in_system label and joining
cur = conn.cursor()
cur.execute("""ALTER TABLE cleaned.remain_in_system ADD COLUMN remains_in_system bool;""")
cur.execute("""ALTER TABLE cleaned.left_system ADD COLUMN remains_in_system bool;""")
cur.execute("""update cleaned.remain_in_system set remains_in_system=true;""")
cur.execute("""update cleaned.left_system set remains_in_system=false;""")
conn.commit()

In [ ]:
cur = conn.cursor()
cur.execute("""create table cleaned.distinct_nie_year_labelled (like cleaned.remain_in_system);""")
cur.execute("""insert into cleaned.distinct_nie_year_labelled select * from cleaned.remain_in_system;""")
cur.execute("""insert into cleaned.distinct_nie_year_labelled select * from cleaned.left_system;""")
conn.commit()

In [ ]:
cur = conn.cursor()
cur.execute("""select a.remains_in_system, b.* into cleaned.student_labelled 
from cleaned.distinct_nie_year_labelled as a 
left join 
cleaned.student_registration as b 
on a.nie=b.nie and a.year=b.year; """)
conn.commit()

In [ ]:
cur = conn.cursor()
cur.execute("""ALTER TABLE cleaned.student_labelled ADD COLUMN dropout bool;""")
cur.execute("""update cleaned.student_labelled as a set dropout=true where a.remains_in_system=false and a.grado_code<12;""")
cur.execute("""update cleaned.student_labelled as a set dropout=false where a.remains_in_system=true and a.grado_code<12;""")
conn.commit()

In [ ]:
col_str

In [ ]:
','.join(renamed_cols)

In [ ]:
col_types

In [ ]:
col_names